# ISMAILA LE GOAT

In [ ]:

import pandas as pd
import numpy as np




In [ ]:
#load data 
df=pd.read_csv("../code/achat_prod_fournisseur_stock.csv") #upload from code file

In [ ]:
df.head()

In [ ]:
# Étape 1 : Correction des anomalies
# Corriger les dates invalides
df['date_achat'] = pd.to_datetime(df['date_achat'], errors='coerce')

# Étape 2 : Vérification et correction des types
df['fiabilité'] = pd.to_numeric(df['fiabilité'], errors='coerce')

# Étape 3 : Création de nouvelles colonnes
df['écart_délai'] = df['délai_livraison_jours'] - df['délai_moyen_jours']

df

In [ ]:
# Statistiques descriptives pour les colonnes numériques
# Import necessary libraries
import pandas as pd

# Select numeric columns and calculate descriptive statistics
numeric_columns = df.select_dtypes(include=['number']).columns
desc_stats_updated = df[numeric_columns].describe()

# Detect outliers using the Interquartile Range (IQR) method
extremes = {}
for col in numeric_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    extremes[col] = {
        'nb_outliers': outliers.shape[0],
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }

# Create a DataFrame for the outliers
extremes_df = pd.DataFrame(extremes).T.sort_values(by='nb_outliers', ascending=False)

# Display the DataFrame (replace ace_tools if unavailable)
print(extremes_df)


les résultats de l'analyse des valeurs extrêmes dans le fichier :

🚩 Variables avec le plus d'outliers :
quantité : 794 valeurs hors de l’intervalle [−87.5, 212.5]

montant_total : 709 valeurs extrêmes

délai_moyen_jours : 262 cas hors norme

prix_unitaire et délai_livraison_jours : aucune valeur extrême détectée selon l'IQR

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Sélection des 3 variables avec le plus d'outliers
top_outlier_cols = extremes_df.head(3).index.tolist()

# Création de boxplots pour visualiser les valeurs extrêmes
plt.figure(figsize=(12, 6))
for i, col in enumerate(top_outlier_cols, 1):
    plt.subplot(1, 3, i)
    sns.boxplot(y=df[col])
    plt.title(f"Boxplot - {col}")
    plt.tight_layout()

plt.suptitle("Visualisation des valeurs extrêmes", y=1.05)
plt.show()


les visualisations des valeurs extrêmes pour les variables :

quantité

montant_total

délai_moyen_jours

Ces boxplots montrent clairement les observations situées bien au-delà des bornes normales (au-dessus des moustaches).

In [ ]:
# Étape 5 : Vérification des valeurs manquantes après conversion
missing_values = df.isnull().sum()

df



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Boîtes à moustaches pour visualiser les distributions et valeurs extrêmes
plt.figure(figsize=(12, 6))
sns.boxplot(data=df[['quantité', 'délai_livraison_jours', 'niveau_stock', 'écart_délai']])
plt.title('Distribution des variables quantitatives avec valeurs extrêmes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

desc_stats, missing_values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Variables pour les différentes prédictions
features_common = ['mois', 'année', 'jour_semaine', 'prix_unitaire', 'fiabilité', 'stock_minimum', 'niveau_stock']
categorical_features = ['catégorie', 'marque', 'pays', 'entrepot']

df


In [ ]:
# Modèle 1 : Prédiction de la quantité achetée
X_quantité = df[features_common + categorical_features]
y_quantité = df['quantité']

# Préparation pipeline avec encodage
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
], remainder='passthrough')

# Split des données
X_train_q, X_test_q, y_train_q, y_test_q = train_test_split(X_quantité, y_quantité, test_size=0.2, random_state=42)

# Pipeline de régression
pipeline_quantité = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

pipeline_quantité.fit(X_train_q, y_train_q)
y_pred_q = pipeline_quantité.predict(X_test_q)
rmse_q = mean_squared_error(y_test_q, y_pred_q, squared=False)

rmse_q

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt

# Suppression de la ligne avec date invalide
df_clean = df.dropna(subset=['date_achat'])

# Encodage des variables catégorielles pertinentes
df_encoded = pd.get_dummies(df_clean, columns=['catégorie', 'marque', 'pays', 'entrepot'], drop_first=True)

# Sélection des features pour prédire la quantité
features_quantité = ['prix_unitaire', 'délai_livraison_jours', 'fiabilité', 'niveau_stock', 'stock_minimum', 'écart_délai']
features_quantité += [col for col in df_encoded.columns if col.startswith(('catégorie_', 'marque_', 'pays_', 'entrepot_'))]

X = df_encoded[features_quantité]
y = df_encoded['quantité']

# Séparation des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Évaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Visualisation : prédictions vs réels
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.4)
plt.xlabel("Quantité réelle")
plt.ylabel("Quantité prédite")
plt.title("Prédiction de la quantité achetée - Régression linéaire")
plt.grid(True)
plt.tight_layout()
plt.show()

mse, r2


La modélisation de la quantité achetée à l'aide d'une régression linéaire donne les résultats suivants :

📈 Résultats de la régression linéaire
MSE (Erreur quadratique moyenne) : 9067.01

R² (coefficient de détermination) : -0.0016

❌ Conclusion : Le modèle n'explique pratiquement rien de la variance (R² ≈ 0), ce qui signifie que la régression linéaire n'est pas adaptée ici. Les prédictions sont très proches de la moyenne, quelle que soit l'entrée

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Entraînement du modèle Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

# Évaluation
rf_mse = mean_squared_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)

# Visualisation des importances des variables
importances = pd.Series(rf_model.feature_importances_, index=X.columns)
importances_sorted = importances.sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=importances_sorted.values[:15], y=importances_sorted.index[:15])
plt.title("Top 15 des variables les plus importantes - Random Forest")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

rf_mse, rf_r2


Le modèle Random Forest donne des résultats légèrement meilleurs mais encore insuffisants :

🌲 Résultats du modèle Random Forest
MSE : 9831.67 (un peu plus élevé que la régression linéaire)

R² : -0.086 → toujours très mauvais (le modèle est pire que la moyenne simple).

📊 Variables les plus importantes :
Le graphique montre les 15 variables ayant le plus de poids dans la prédiction. Parmi les plus influentes, on retrouve généralement :

prix_unitaire

niveau_stock

stock_minimum

Certaines catégories ou marques spécifiques

❓Interprétation
La variable cible "quantité achetée" semble difficilement prévisible à partir des données disponibles. Il est possible que :

D'autres facteurs importants soient absents (budgets, promotions, saisonnalité fine, etc.).

Ou que la variable soit sujette à une grande variabilité aléatoire.

In [ ]:
# Modèle 2 : Prédiction du délai de livraison
X_délai = df[features_common + categorical_features]
y_délai = df['délai_livraison_jours']

# Split des données
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_délai, y_délai, test_size=0.2, random_state=42)

# Pipeline de régression
pipeline_délai = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

pipeline_délai.fit(X_train_d, y_train_d)
y_pred_d = pipeline_délai.predict(X_test_d)
rmse_d = mean_squared_error(y_test_d, y_pred_d, squared=False)

rmse_d


In [ ]:
# Suppression des colonnes non numériques non encodées
cols_to_drop = ['délai_livraison_jours', 'quantité', 'montant_total', 'date_achat', 'id_achat',
                'id_produit', 'id_fournisseur', 'nom_fournisseur', 'ville']
X_delay_cleaned = df_encoded.drop(columns=cols_to_drop)
y_delay = df_encoded['délai_livraison_jours']

# Séparation train/test
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_delay_cleaned, y_delay, test_size=0.2, random_state=42)

# Entraînement Random Forest
rf_delay_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_delay_model.fit(X_train_d, y_train_d)
y_pred_delay = rf_delay_model.predict(X_test_d)

# Évaluation
mse_delay = mean_squared_error(y_test_d, y_pred_delay)
r2_delay = r2_score(y_test_d, y_pred_delay)

# Visualisation : Prédiction vs Réel
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test_d, y=y_pred_delay, alpha=0.4)
plt.xlabel("Délai réel (jours)")
plt.ylabel("Délai prédit (jours)")
plt.title("Prédiction du délai de livraison fournisseur")
plt.grid(True)
plt.tight_layout()
plt.show()

mse_delay, r2_delay


La prédiction du délai de livraison fournisseur avec le modèle Random Forest est très performante :

✅ Résultats du modèle
MSE : 0.0041 → très faible erreur moyenne.

R² : 0.9997 → le modèle explique quasiment toute la variance des délais.

📊 Interprétation
Les données disponibles contiennent des variables très corrélées au délai de livraison (probablement délai_moyen_jours, fiabilité, ou certaines catégories/fournisseurs).

In [ ]:
# Modèle 3 : Classification des fournisseurs selon leur pays
# Pour cela on utilise les caractéristiques liées au fournisseur

X_pays = df[['fiabilité', 'délai_moyen_jours', 'stock_minimum', 'niveau_stock', 'entrepot', 'ville']]
y_pays = df['pays']

# Encodage des variables catégorielles
categorical_pays_features = ['entrepot', 'ville']
preprocessor_pays = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_pays_features)
], remainder='passthrough')

# Split
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_pays, y_pays, test_size=0.2, random_state=42)

# Pipeline
pipeline_pays = Pipeline(steps=[
    ('preprocessor', preprocessor_pays),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_pays.fit(X_train_p, y_train_p)
y_pred_p = pipeline_pays.predict(X_test_p)
report_pays = classification_report(y_test_p, y_pred_p, output_dict=True)

# Conversion du rapport en DataFrame
report_pays_df = pd.DataFrame(report_pays).transpose()

# Affichage du DataFrame
print("Rapport Classification Fournisseurs par Pays")
print(report_pays_df)

# Matrice de confusion
cm = confusion_matrix(y_test_c, y_pred_c, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot(xticks_rotation=45, cmap='Blues')
plt.title("Matrice de confusion - Classification du pays du fournisseur")
plt.tight_layout()
plt.show()

La classification du pays du fournisseur donne d’excellents résultats pour les pays les plus représentés :

🎯 Performances (extraits)
Allemagne : Précision 93.7%, Recall 97.4%, F1-score 95.5%

Finlande : Précision 97.5%, Recall 99.8%, F1-score 98.6%

Les petits pays (comme Espagne, Danemark) ont des performances plus faibles à cause d’un nombre d’exemples limité.

🔎 Le modèle est très fiable pour les pays majoritaires, mais pourrait être amélioré pour les pays minoritaires via du rééquilibrage.

In [ ]:
# Modèle 4 : Analyse des niveaux de stock par entrepôt
# On va prédire le niveau de stock selon entrepôt (modèle de régression)

X_stock = df[['mois', 'année', 'jour_semaine', 'prix_unitaire', 'quantité', 'fiabilité', 'stock_minimum', 'entrepot']]
y_stock = df['niveau_stock']

categorical_stock_features = ['entrepot']
preprocessor_stock = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_stock_features)
], remainder='passthrough')

# Split
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_stock, y_stock, test_size=0.2, random_state=42)

# Pipeline
pipeline_stock = Pipeline(steps=[
    ('preprocessor', preprocessor_stock),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

pipeline_stock.fit(X_train_s, y_train_s)
y_pred_s = pipeline_stock.predict(X_test_s)
rmse_s = mean_squared_error(y_test_s, y_pred_s, squared=False)

rmse_s



In [ ]:
# Boîte à moustaches : distribution du niveau de stock par entrepôt
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_clean, x='entrepot', y='niveau_stock')
plt.title("Distribution des niveaux de stock par entrepôt")
plt.xlabel("Entrepôt")
plt.ylabel("Niveau de stock")
plt.grid(True)
plt.tight_layout()
plt.show()

# Moyenne du stock par entrepôt
stock_by_entrepot = df_clean.groupby('entrepot')['niveau_stock'].agg(['mean', 'median', 'std', 'count']).sort_values('mean', ascending=False)


 l'analyse des niveaux de stock par entrepôt :

📊 Visualisation
Les entrepôts Paris, Lyon, et Marseille montrent des distributions similaires mais avec quelques différences de médiane et de dispersion.

Des valeurs extrêmes sont visibles dans chaque entrepôt.

📈 Statistiques principales
Paris a le niveau moyen de stock le plus élevé (~150.6).

Lyon est proche (~147.6), mais avec une plus grande dispersion.

Marseille a le niveau moyen le plus bas (~139.0) et la plus petite médiane.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extraire les importances des features
feature_names = pipeline_stock.named_steps['preprocessor'].get_feature_names_out().tolist() + [
    f for f in X_stock.columns if f not in categorical_stock_features
]
importances = pipeline_stock.named_steps['regressor'].feature_importances_
indices = np.argsort(importances)[::-1]

# Tracer l'importance des variables
plt.figure(figsize=(10, 6))
plt.title("Importance des variables pour la prédiction du niveau de stock")
plt.bar(range(len(importances)), importances[indices], align='center')
plt.xticks(range(len(importances)), np.array(feature_names)[indices], rotation=90)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Moyennes des niveaux de stock par entrepôt
stock_features = df_clean.groupby('entrepot')[['niveau_stock']].mean()

# Standardisation
scaler = StandardScaler()
stock_scaled = scaler.fit_transform(stock_features)

# Clustering avec KMeans (choix de 3 clusters pour tester)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
stock_features['cluster'] = kmeans.fit_predict(stock_scaled)

# Visualisation
plt.figure(figsize=(8, 6))
sns.barplot(x=stock_features.index, y=stock_features['niveau_stock'], hue=stock_features['cluster'], dodge=False)
plt.title("Segmentation des entrepôts par niveau moyen de stock")
plt.ylabel("Niveau moyen de stock")
plt.xlabel("Entrepôt")
plt.tight_layout()
plt.show()




 la segmentation des entrepôts selon leur niveau moyen de stock à l’aide de KMeans :

🧠 Clustering
Les entrepôts ont été répartis en 3 clusters distincts.

Cela permet d’identifier des sites avec un stock élevé, moyen ou faible.

On peut utiliser cette segmentation pour adapter les politiques de réassort, de capacité, ou pour équilibrer les flux logistiques.

In [ ]:
import ace_tools 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, top_k_accuracy_score
import numpy as np

# Nettoyage de la date
df['date_achat'] = pd.to_datetime(df['date_achat'], errors='coerce')
df = df.dropna(subset=['date_achat'])  # supprimer les lignes avec dates invalides

# Feature engineering
df['écart_délai'] = df['délai_livraison_jours'] - df['délai_moyen_jours']

# Suppression des colonnes inutiles
drop_cols = ['id_achat', 'date_achat', 'id_produit', 'nom_fournisseur', 'ville']
df_features = df.drop(columns=drop_cols)

# Encodage de la variable cible
le = LabelEncoder()
y = le.fit_transform(df['id_produit'])

# Encodage des variables catégorielles restantes
X = pd.get_dummies(df_features, drop_first=True)

# Séparation des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement du modèle
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Prédiction
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)


# Provide all known classes to the `labels` parameter
all_classes = clf.classes_  # Extract all classes from the trained classifier
top_3_accuracy = top_k_accuracy_score(y_test, y_proba, k=3, labels=all_classes)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Display the classification report
print("Rapport de classification - Produits")
print(report_df)

# Optionally, save the report to a CSV file for further analysis
report_df.to_csv("classification_report_produits.csv", index=True)

# Return accuracy and top-3 accuracy
accuracy, top_3_accuracy

In [ ]:
! pip install lightgbm

In [ ]:
# Utilisation de LightGBM pour classifier les produits parmi le top 50
import lightgbm as lgb

# Création du dataset LightGBM
lgb_train = lgb.Dataset(X_train_top, label=y_train_top)
lgb_eval = lgb.Dataset(X_test_top, label=y_test_top, reference=lgb_train)

# Paramètres de base pour multiclass
params = {
    'objective': 'multiclass',
    'num_class': len(np.unique(y_train_top)),
    'metric': 'multi_logloss',
    'verbosity': -1,
    'seed': 42
}

# Entraînement
lgb_model = lgb.train(params, lgb_train, valid_sets=[lgb_eval], num_boost_round=100, early_stopping_rounds=10, verbose_eval=False)

# Prédictions
y_proba_lgb = lgb_model.predict(X_test_top)
y_pred_lgb = np.argmax(y_proba_lgb, axis=1)

# Évaluation
accuracy_lgb = accuracy_score(y_test_top, y_pred_lgb)
top3_accuracy_lgb = top_k_accuracy_score(y_test_top, y_proba_lgb, k=3, labels=np.arange(y_proba_lgb.shape[1]))

accuracy_lgb, top3_accuracy_lgb
